In [13]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import skimage
from PIL import Image
plt.figure(figsize=(20, 20))
import pandas as pd
import io
import torch
from VGGModel import VGGModel
from torchvision import transforms


<Figure size 1440x1440 with 0 Axes>

In [76]:
class Inferance:
    def __init__(self,Model_path):
        """ Make prediction for given Audio File """
        self.model = VGGModel().Build_Model()
        self.model.load_state_dict(torch.load(Model_path, map_location=torch.device('cpu')))
        self.model.eval() 
        self.langs= ['ar', 'de', 'en', 'es', 'fr', 'it', 'pt']


    def _Generate_MelSpec(self, audio_path):
        """Generate Mel_spectogram image from Audio file"""
        y, sr = librosa.load(audio_path ,duration = 8, sr=16000)
        y = librosa.util.fix_length(y, size=8*16000)
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=sr, fmin=20)
        
        img = librosa.display.specshow(librosa.power_to_db(S, ref=np.max),
                                    x_axis='time', y_axis='mel', fmax=sr )
        plt.axis("off")


        buf = io.BytesIO()
        plt.savefig(buf, format='jpg')
        buf.seek(0)
        image = Image.open(buf).resize((224, 224))

        # img_array = np.array(image)

        buf.close()
        plt.close()

        return image


    def Predict(self, Image_Path):
        PIL_img = self._Generate_MelSpec(Image_Path)
        pil_to_tensor = transforms.ToTensor()(PIL_img).unsqueeze_(0)
        prediction = self.model(pil_to_tensor)
        _,Pred =torch.max(prediction, dim=1)
        # prediction = self.model(pil_to_tensor)
        return self.langs[Pred.numpy()[0]]

In [79]:
infer = Inferance("VGG_Mel128_200_M_12088_27801.pt")


'ar'

In [85]:
print(infer.Predict("../../../../../DataSets/FinalDataSet/clips/ar-f-0-10-CV.wav"))
print(infer.Predict("../../../../../DataSets/FinalDataSet/clips/de-n-6-6-VF.wav"))
print(infer.Predict("../../../../../DataSets/FinalDataSet/clips/en-n-33-4-VF.wav"))
print(infer.Predict("../../../../../DataSets/FinalDataSet/clips/es-n-5-0-VF.wav"))
print(infer.Predict("../../../../../DataSets/FinalDataSet/clips/fr-n-1731-0-VF.wav"))
print(infer.Predict("../../../../../DataSets/FinalDataSet/clips/it-n-2-3-VF.wav"))
print(infer.Predict("../../../../../DataSets/FinalDataSet/clips/pt-m-158-10145-CV.wav"))

ar
de
en
es
en
es
es
